In [ ]:
import importlib
from dasilva_invariants import meshes, invariants
importlib.reload(meshes)
importlib.reload(invariants)

import pylab as plt
import numpy as np
%matplotlib inline

mesh = meshes.get_lfm_hdf4_data('../lfm_data/hdf4/March2015_mhd_2015-03-17T04-45-00Z.hdf')

In [ ]:
r = 6
th = 1
x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r

result = invariants.calculate_K(mesh, (x, y, 0), 7.5, step_size=None)

In [ ]:
mask = result.trace_field_strength < result.Bm

plt.figure(figsize=(8, 5))
plt.scatter(x=result.trace_points[:, 0],
            y=result.trace_points[:, 2],
            c=np.log10(result.trace_field_strength))
#plt.plot(result.trace_points[mask, 0],
#         result.trace_points[mask, 2],
#         'rx', markersize=20, label='B < Bm')
cb = plt.colorbar()
cb.set_label('Log10(B) (Log-Gauss)')
plt.xlabel('X (Re)')
plt.ylabel('Z (Re)')
plt.legend()
plt.grid(color='#ccc', linestyle='dashed')
plt.title('Field Line Trace')

In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(result.integral_axis_latitude, result.integral_integrand, 'k,-')
plt.fill_between(result.integral_axis_latitude,
                 result.integral_integrand.min(),
                 result.integral_integrand)
plt.title(f'K = {result.K:.6f} Re Sqrt(G) = ' '$\int_{s_m}^{s_m\'}\sqrt{B_m - B(s)}ds$\n'
          f'Bm = {result.Bm:.6f} G\n'
          f'Mirror at {result.integral_axis_latitude.max():.1f} deg',
          fontsize=20)
plt.xlabel('Magnetic Latitude (deg)', fontsize=20)
plt.ylabel('$\sqrt{B_m - B(s)}$', fontsize=20)
None

In [ ]:
plt.figure(figsize=(10, 7), dpi=300)
rs = np.arange(3, 8)
th = 0
cmap = plt.get_cmap('viridis')

for i, r in enumerate(rs):
    x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
    result = invariants.calculate_K(mesh, (r, 0, 0), 7.5, step_size=None)
    rel_position = np.arange(result.trace_latitude.size) /  result.trace_latitude.size
    
    plt.plot(rel_position, result.trace_field_strength, ',-', label=f'r={r}', color=cmap(i/rs.size))
    plt.xlabel('Relative Position in Trace')
    
plt.ylabel('|B| (Gauss)')
plt.title(f'Trace from (x,y,z)=(r,0,0) with Bm at 7.5 deg')
plt.yscale('log')
plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(color='#ccc', linestyle='dashed')

In [ ]:
importlib.reload(invariants)

rs = np.linspace(3, 6, 100)
Ks = np.zeros_like(rs)

for i, r in enumerate(rs):
    x, y = np.cos(np.deg2rad(10)) * r, np.sin(np.deg2rad(10)) * r
    Ks[i] = invariants.calculate_K(mesh, (x, y, 0), 7.5).K
    
plt.plot(rs, Ks, '.-')
plt.title('K vs L-Shell on LFM Grid', fontsize=20)
plt.xlabel('L-Shell', fontsize=15)
plt.ylabel('K(L)', fontsize=15)

In [ ]:
r = 3
th = 1
x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r

result = invariants.calculate_LStar(mesh, (x, y, 0), 7.5, num_local_times=20,
                                    rel_error_threshold=0.01, verbose=True)

In [ ]:
localtimes = np.rad2deg(result.drift_local_times) * 24 / 360 + 12
localtimes %= 24

I = np.argsort(localtimes)

plt.figure()
plt.plot(localtimes[I], result.drift_lvalues[I], '.-')
plt.xlabel('Local time')
plt.ylabel('L-Shell')

plt.figure()
plt.plot(localtimes[I], result.drift_K[I], '.-')
plt.xlabel('Local time')
plt.ylabel('K')

In [ ]:
import glob
files = glob.glob('../lfm_data/hdf4/*.hdf')[:10]

results = []
for file in files:
    result = None
    mesh = meshes.get_lfm_hdf4_data(file)
    try:
        result = invariants.calculate_LStar(mesh, (4, 0, 0), 7.5, num_local_times=30,
                                        rel_error_threshold=0.03, verbose=True)
        if result:
            results.append((file, result))
    except RuntimeError:
        pass

In [ ]:
plt.figure(figsize=(12, 8))
#plt.gca().set_facecolor('#ccc')
plt.xlabel('Local time')
plt.ylabel('L-Shell')

cmap = plt.get_cmap('viridis')

for i, (file, result) in enumerate(results):
    localtimes = np.rad2deg(result.drift_local_times) * 24 / 360 + 12
    localtimes %= 24
    I = np.argsort(localtimes)
    plt.plot(localtimes[I], result.drift_lvalues[I], '.-', label=file.split('/')[-1], color=cmap(i/len(results)))

plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

In [ ]:
plt.figure(figsize=(8, 8))

cmap = plt.get_cmap('viridis')

for i, (file, result) in enumerate(results):
    x = np.cos(result.drift_local_times) * result.drift_lvalues
    y = np.sin(result.drift_local_times) * result.drift_lvalues
    
    x = x.tolist() + [x[0]]
    y = y.tolist() + [y[0]]
    
    plt.plot(x, y, color=cmap(i/len(results)), label=file.split('/')[-1])

plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5))
#plt.yscale('symlog')
#plt.xscale('symlog')
plt.grid()

In [ ]:
plt.plot(results[0][1].drift_local_times, results[0][1].drift_lvalues)

In [ ]:
import h5py
hdf = h5py.File('rbspa_def_MagEphem_OP77Q_20150317_v3.0.0.h5')
hdf.keys()

In [ ]:
import dateutil.parser
iso_time = np.array([dateutil.parser.parse(t) for t in hdf['IsoTime'][:]])
Rsm = hdf['Rsm'][:]
dst = hdf['Dst'][:]
iso_time

In [ ]:
import glob
files = glob.glob('../lfm_data/hdf4/*.hdf')

def coerce_string(f):
    return f.split('_')[-1][:-4].replace('-', ':').replace(':', '-', 2)

file_times = np.array([dateutil.parser.parse(coerce_string(f)) for f in files])
#mesh = meshes.get_lfm_hdf4_data('../lfm_data/hdf4/March2015_mhd_2015-03-17T04-45-00Z.hdf')
file_times[:3]

In [ ]:
import progressbar
Ks = []

bar = progressbar.ProgressBar()
rs = []

for file, file_time in bar(list(zip(files, file_times))):
    x, y, z = Rsm[iso_time.searchsorted(file_time)]
    r = np.sqrt(x**2 + y**2 + z**2)
    
    rs.append(r)
    
    if r < 3:
        Ks.append(np.nan)
        continue
        
    mesh = meshes.get_lfm_hdf4_data(file)
    try:
        result = invariants.calculate_K(mesh, (x, y, z), 7.5, step_size=None)
    except:
        raise RuntimeError((x, y, z, r))
    Ks.append(result.K)

In [ ]:
tmp = file_times[~np.isnan(Ks)]
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(6, 9))
axes[0].plot(file_times, np.array(Ks)*1000, 'o-')
axes[0].set_ylabel('K (milli-ReSqrt(G))')
axes[1].plot(file_times, rs, 'ro-')
axes[1].set_ylabel('RSBSP A Radius (Re)')
axes[1].set_xlim(tmp[0], tmp[-1])
axes[2].plot(iso_time, dst, 'ko-')
axes[2].set_ylim([0, 50])
axes[2].set_ylabel('Dst')

for ax in axes:
    ax.grid(color='#ccc', linestyle='dashed')